In [1]:
from peft import PeftModel, PeftConfig

In [2]:
!pip install bitsandbytes

In [3]:
checkpoint_model = "manojbaniya/best_01"

In [8]:
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained(checkpoint_model, cache_dir="./model")
model = AutoModelForCausalLM.from_pretrained(checkpoint_model, load_in_8bit=True, device_map='auto', cache_dir="./model")

tokenizer_config.json:   0%|          | 0.00/46.4k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/34.4M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.37k [00:00<?, ?B/s]

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.
Unused kwargs: ['_load_in_4bit', '_load_in_8bit', 'quant_method']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.
/usr/local/lib/python3.11/dist-packages/transformers/quantizers/auto.py:186: UserWarning: You passed `quantization_config` or equivalent parameters to `from_pretrained` but the model you're loading already has a `quantization_config` attribute. The `quantization_config` from the model will be used.
  warnings.warn(warning_msg)


model.safetensors:   0%|          | 0.00/2.22G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [5]:
prompt = """
Below is an instruction that describes a task paired with an input that provides further context. Write a response that appriately complete the request.

### instruction:
{instruction}

### question:
{question}

### input:
{context}

### response:
{response}
"""

In [13]:
input = prompt.format(
    instruction="talako user ko question lai context herera answer vana",
    question="store ko location kaha ho",
    context="Hamro store name all electronics store ho ra hamro store Dharna ma xa hami esewa khalti baata payment accept garxau",
    response=""
)

In [9]:
inputs = tokenizer(input, return_tensors="pt").to("cuda")
inputs["input_ids"]

tensor([[     2,    108,  33501,    603,    671,  14239,    674,  19306,    476,
           6911,  44600,    675,    671,   3772,    674,   6572,   4024,   4807,
         235265,  15615,    476,   3590,    674, 207546,   2823,   3407,    573,
           3853, 235265,    109,   6176,  14239, 235292,    108,   3559,  12841,
           2425,   5778,   2872,  50704,   4807,   1070,   1932,   3448, 212193,
            109,   6176,   2872, 235292,    108,   6709,   5778,   5201,   5675,
            527,   1965,    109,   6176,   3772, 235292,    108,  20949,    514,
           4659,   1503,    832,  36147,   4659,   1965,   1051,  12578,    514,
           4659,  91995,    556,    718,  50521,    109,   6176,   3590, 235292,
            109]], device='cuda:0')

In [10]:
outputs = model.generate(**inputs, max_new_tokens=512)

The 'batch_size' attribute of HybridCache is deprecated and will be removed in v4.49. Use the more precisely named 'self.max_batch_size' attribute instead.


In [11]:
print(tokenizer.decode(outputs[0], skip_special_tokens=True))


Below is an instruction that describes a task paired with an input that provides further context. Write a response that appriately complete the request.

### instruction:
talako user ko question lai context herera answer vana

### question:
store ko location kaha ho

### input:
Hamro store name all electronics store ho ra hamro store Dharna ma xa

### response:

Hamro store Dharna ma ho ra hamro location Dharna ma xa.



In [27]:
def generate_answer(question: str) -> str:
    input = prompt.format(instruction="user le gareko question lai answer deu",
    question="store ko location kaha ho",
    context="Hamro store name all electronics store ho ra hamro store Dharna ma xa hami esewa khalti baata payment accept garxau",
    response="")
    inputs = tokenizer(input, return_tensors="pt").to("cuda")
    outputs = model.generate(**inputs, max_new_tokens=512)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

In [28]:
generate_answer("payment method ke ke ho")

'\nBelow is an instruction that describes a task paired with an input that provides further context. Write a response that appriately complete the request.\n\n### instruction:\nuser le gareko question lai answer deu\n\n### question:\nstore ko location kaha ho\n\n### input:\nHamro store name all electronics store ho ra hamro store Dharna ma xa hami esewa khalti baata payment accept garxau\n\n### response:\n\nStore Dharna ma ho.\n'

In [30]:
def get_inputs(query, prompt=prompt):
  instruction = "User le gareko question lai answer deu"
  question = query
  context = ""
  response = ""
  inputs = prompt.format(instruction=instruction, question=question, context=context, response=response)
  inputs = tokenizer(inputs, return_tensors="pt").to("cuda")
  return inputs

In [31]:
import torch

In [46]:
def generate_tokens(query, max_new_tokens=100):
    inputs = get_inputs(query)
    is_done = False
    tokens = []
    text = []
    while not is_done:
        with torch.no_grad():
            outputs = model(inputs["input_ids"])
            logits = outputs.logits[:, -1, :]
            preds = logits.softmax(dim=1)
            label = preds.argmax(dim=-1)
            print(tokenizer.decode(label))
            text.append(tokenizer.decode(label))
            # Convert label to a Python integer before appending
            tokens.append(label.item())
            inputs["input_ids"] = torch.cat([inputs["input_ids"], label.unsqueeze(-1)], dim=-1)
            if label == tokenizer.eos_token_id or len(tokens) >= max_new_tokens:
                is_done = True
    return tokenizer.decode(tokens)

In [37]:
generate_tokens("Nepal ko national language lai lai vaninxa?")

Nepal
 ko
 national
 language
 Nepali
 ho
.


<eos>


'Nepal ko national language Nepali ho.\n<eos>'

In [47]:
generate_tokens("kasari good habit develop garne")

Good
 habits
 develop
 gar
na
 daily
 routine
 ma
 focus
 gar
na
,
 positive
 thinking
 ra
 positive
 vibes
 har
u
,
 ra
 daily
 goals
 set
 gar
na
 par
cha
.


<eos>


'Good habits develop garna daily routine ma focus garna, positive thinking ra positive vibes haru, ra daily goals set garna parcha.\n<eos>'

In [48]:
generate_tokens("kasari healthy diet maintain garne")

Healthy
 diet
 maintain
 gar
na
 fresh
 fruits
,
 vegetables
,
 whole
 grains
,
 lean
 proteins
,
 and
 healthy
 fats
 ra
 healthy
 lifestyle
 follow
 gar
na
 par
cha
.


<eos>


'Healthy diet maintain garna fresh fruits, vegetables, whole grains, lean proteins, and healthy fats ra healthy lifestyle follow garna parcha.\n<eos>'

In [49]:
generate_tokens("coding kasari sikna sakinxa")

Coding
 language
 ko
 basics
,
 data
 structures
,
 algorithms
,
 and
 programming
 language
 ko
 use
 learn
 gar
na
 sak
n
uh
os
.


<eos>


'Coding language ko basics, data structures, algorithms, and programming language ko use learn garna saknuhos.\n<eos>'

In [51]:
generate_tokens("india ko prime minister ko name ke ho")

india
 ko
 prime
 minister
 ko
 name
 nare
ndra
 modi
 ho
.


<eos>


'india ko prime minister ko name narendra modi ho.\n<eos>'

In [59]:
generate_tokens("talako context herera answer vana: hamro store ko name all happy store ho ra yo Dharan ma xa. ### question: store ko location kaha xa ### reponse: ")

D
haran
 ma
 all
 happy
 store
 ko
 location
 Dhar
an
 ma
 ho
.


<eos>


'Dharan ma all happy store ko location Dharan ma ho.\n<eos>'

In [60]:
generate_tokens("diyeko words baata odd one identify gara:apple, cat, dog, mouse")

apple
 odd
 one
 ho
,
 ani
 ani
 words
 ko
 context
 ma
 apple
 ko
 relevance
 ani
 ani
 words
 ko
 relation
 ani
 ani
 context
 ma
 odd
 one
 identify
 gar
na
 par
cha
.


<eos>


'apple odd one ho, ani ani words ko context ma apple ko relevance ani ani words ko relation ani ani context ma odd one identify garna parcha.\n<eos>'

In [62]:
generate_tokens("ke baata payment garna sakinxa")

Payment
 sakin
xa
 online
 payment
 ma
 hun
cha
.


<eos>


'Payment sakinxa online payment ma huncha.\n<eos>'

In [63]:
generate_tokens("Nepal ma katiwata pradesh xa")

Nepal
 ma
 Uttar
 Pradesh
 pra
desh
 cha
.


<eos>


'Nepal ma Uttar Pradesh pradesh cha.\n<eos>'

### serving

In [64]:
!pip install flask

In [66]:
!pip install pyngrok

In [80]:
ngrok.set_auth_token("2riLb0UtZa6Vf4YtDV7fdJExpKu_5njCcgcsSrLJnKfw8gXP6")

In [88]:
from pyngrok import ngrok

# Start the Flask server
public_url = ngrok.connect(5000)
print("Public URL:", public_url)

Public URL: NgrokTunnel: "https://64c2-34-16-160-231.ngrok-free.app" -> "http://localhost:5000"


In [89]:
from flask import Flask, request, jsonify

app = Flask(__name__)

@app.route('/')
def home():
    return "Hello, this is your server running on Google Colab!"

@app.route('/chat', methods=['POST'])
def chat():
    # Get the user's query from the request
    user_query = request.json.get('query', '')

    # Process the query (simple example response)
    if user_query:
        response = generate_tokens(user_query)
    else:
        response = "Please provide a valid query."

    # Return a JSON response
    return jsonify({"response": response})

if __name__ == '__main__':
    app.run(host='localhost', port=5000)


 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://localhost:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [16/Jan/2025 14:27:35] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [16/Jan/2025 14:27:36] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [16/Jan/2025 14:27:59] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [16/Jan/2025 14:28:05] "GET / HTTP/1.1" 200 -


Nepal
 ko
 prime
 minister
 ko
 Sher
 Bahadur
 De
uba
 ho
.




INFO:werkzeug:127.0.0.1 - - [16/Jan/2025 14:28:14] "POST /chat HTTP/1.1" 200 -


<eos>
Nepal
 ko
 prime
 minister
 ko
 Sher
 Bahadur
 De
uba
 ho
.




...

<eos>


INFO:werkzeug:127.0.0.1 - - [16/Jan/2025 14:36:47] "POST /chat HTTP/1.1" 400 -


Lionel
 Messi
 is
 a
 professional
 footballer
 who
 plays
 for
 Paris
 Saint
-
Germain
 in
 France
 and
 the
 Argentina
 national
 team
.
 He
 is
 widely
 regarded
 as
 one
 of
 the
 greatest
 players
 of
 all
 time
 and
 has
 won
 numerous
 individual
 and
 team
 awards
 throughout
 his
 career
.




INFO:werkzeug:127.0.0.1 - - [16/Jan/2025 14:37:02] "POST /chat HTTP/1.1" 200 -


<eos>


## stream token

In [93]:
def generate_tokens(query, max_new_tokens=100):
    inputs = get_inputs(query)
    is_done = False
    tokens = []

    while not is_done:
        with torch.no_grad():
            outputs = model(inputs["input_ids"])
            logits = outputs.logits[:, -1, :]
            preds = logits.softmax(dim=1)
            label = preds.argmax(dim=-1)

            # Decode and yield the token
            decoded_token = tokenizer.decode(label)
            yield decoded_token  # Yield the decoded token one by one
            print(decoded_token)
            # Append token to the list
            tokens.append(label.item())

            # Update the input_ids for the next iteration
            inputs["input_ids"] = torch.cat([inputs["input_ids"], label.unsqueeze(-1)], dim=-1)

            # Check if generation is done
            if label == tokenizer.eos_token_id or len(tokens) >= max_new_tokens:
                is_done = True

In [96]:
from pyngrok import ngrok

# Start the Flask server
public_url = ngrok.connect(5000)
print("Public URL:", public_url)

Public URL: NgrokTunnel: "https://bd1d-34-16-160-231.ngrok-free.app" -> "http://localhost:5000"


In [ ]:
from flask import Flask, request, jsonify, Response

app = Flask(__name__)

@app.route('/')
def home():
    return "Hello, this is your server running on Google Colab!"

@app.route('/chat', methods=['POST'])
def chat():
    # Get the user's query from the request
    user_query = request.json.get('query', '')

    if not user_query:
        # Return a single JSON response if query is missing
        return jsonify({"response": "Please provide a valid query."})

    def generate_response():
        # Call the generator function and yield tokens as they are generated
        for token in generate_tokens(user_query):
            print(f"Sent: {token}")
            yield f"data: {token}\n\n"  # Format for server-sent events (SSE)

    # Return a streaming response with the correct content type for SSE
    return Response(generate_response(), content_type='text/event-stream')

if __name__ == '__main__':
    app.run(host='localhost', port=5000)

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://localhost:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [16/Jan/2025 14:45:23] "POST /chat HTTP/1.1" 200 -


Sent: Nepal
Nepal
Sent:  ko
 ko
Sent:  capital
 capital
Sent:  city
 city
Sent:  Kathmandu
 Kathmandu
Sent:  ho
 ho
Sent: .
.
Sent: 



Sent: <eos>
<eos>


INFO:werkzeug:127.0.0.1 - - [16/Jan/2025 14:45:43] "POST /chat HTTP/1.1" 200 -


Sent: Nepal
Nepal
Sent:  ko
 ko
Sent:  capital
 capital
Sent:  city
 city
Sent:  Kathmandu
 Kathmandu
Sent:  ho
 ho
Sent: .
.
Sent: 



Sent: <eos>
<eos>


INFO:werkzeug:127.0.0.1 - - [16/Jan/2025 14:46:38] "POST /chat HTTP/1.1" 200 -


Sent: Nepal
Nepal
Sent:  ko
 ko
Sent:  capital
 capital
Sent:  city
 city
Sent:  Kathmandu
 Kathmandu
Sent:  ho
 ho
Sent: .
.
Sent: 



Sent: <eos>
<eos>


INFO:werkzeug:127.0.0.1 - - [16/Jan/2025 14:47:20] "POST /chat HTTP/1.1" 200 -


Sent: Nepal
Nepal
Sent:  ko
 ko
Sent:  capital
 capital
Sent:  city
 city
Sent:  Kathmandu
 Kathmandu
Sent:  ho
 ho
Sent: .
.
Sent: 



Sent: <eos>
<eos>
